In [1]:
import torch

from torch import nn
from torch.utils.data import Dataset,DataLoader
from torchvision.transforms import Compose
from torch.optim import SGD

import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_breast_cancer

In [2]:
class Cancer_Dataset(Dataset):
    
    def __init__(self,transforms=None):
        X,y = load_breast_cancer(return_X_y=True)
        
        self.mu = torch.tensor(np.mean(X,0),dtype=torch.float32).view(1,-1)
        self.sd = torch.tensor(np.std(X,0),dtype=torch.float32).view(1,-1)
        
        self.x = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)
        
        self.transforms = transforms
        self.len = X.shape[0]
        
        print("Dataset Constructed")
        
    def __getitem__(self,index):
        sample = self.x[index],self.y[index]
        
        if self.transforms:
            sample = self.transforms(sample)
        
        return sample
    
    def __len__(self):
        return self.len

In [3]:
class Standard_Normalizer(object):
    def __init__(self,mu,std):
        self.mu = mu
        self.std = std
        
        print("Standard Normalizer Constructed")
        
    def __call__(self,sample):
        x = sample[0]
        y = sample[1]
        
        x = (x-self.mu)/self.std
        sample = x,y
        return sample

In [4]:
class Logistic_Regression(nn.Module):
    
    def __init__(self,num_input=1,num_classes=2):
        super(Logistic_Regression,self).__init__()
        self.Linear_Regression = nn.Linear(num_input,num_classes)
        self.Softmax = nn.Softmax(2)
        
    def forward(self,x):
        z = self.Linear_Regression(x)
        y = self.Softmax(z)
        return y

In [5]:
criterion = torch.nn.NLLLoss()

In [6]:
dataset = Cancer_Dataset()
normalizer = Standard_Normalizer(dataset.mu,dataset.sd)
dataset.transforms = normalizer

Dataset Constructed
Standard Normalizer Constructed


In [7]:
model = Logistic_Regression(30,2)

In [8]:
batch_size = 32
dataloader = DataLoader(dataset=dataset,batch_size=batch_size)

In [9]:
lr = 0.1
optimizer = SGD(model.parameters(),lr=lr)

In [10]:
print_every = 1

for epoch in range(10):
    l = []
    for x,y in dataloader:
        y_pred = model(x)
        
        loss = criterion(y_pred.squeeze(),y)
        l.append(loss.detach().item())

        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        

    if (epoch+1) % print_every == 0:
        print("Epoch {}: loss={}".format(epoch+1,np.mean(l)))

Epoch 1: loss=-0.7502217690149943
Epoch 2: loss=-0.8834092451466454
Epoch 3: loss=-0.9076853758758969
Epoch 4: loss=-0.9198380808035532
Epoch 5: loss=-0.9278127219941881
Epoch 6: loss=-0.9338585899935828
Epoch 7: loss=-0.938512020640903
Epoch 8: loss=-0.9420249462127686
Epoch 9: loss=-0.9447945422596402
Epoch 10: loss=-0.9470566444926791


In [11]:
with torch.no_grad():
    predictions = []
    for x,y in dataset:
        y_pred = model(x.unsqueeze(0))
        predictions.append(y_pred.argmax().item())

In [12]:
print("Accuracy",np.mean(dataset.y.numpy() == np.array(predictions)))

Accuracy 0.984182776801406
